In [ ]:
import numpy as np
from scipy.linalg import eig
import matplotlib.pylab as plt
from matplotlib import cm
from numpy import sin, cos, tan, exp, pi, e

# Defining the variables and equations of motion

In [ ]:
X_LABEL = "x"
Y_LABEL = "y"

def DXDT(x, y):
    return(2*x + 3*y)
    
def DYDT(x, y):
    return(2*x + 1*y)

# Rendering the phase portrait

In [ ]:
# Setting plotting defaults 
X_MIN = -6
X_MAX = 6
Y_MIN = -6
Y_MAX = 6
RES = 0.5
Q_SCALE = 2.0

In [ ]:
# Finding critical points

from scipy.optimize import fsolve
from itertools import product

## Needed for critical point solver, DO NOT CHANGE
def DDT(arg):
    x, y = arg
    return(DXDT(x, y), DYDT(x, y))

#
epsilon = 0.00001
res_c = 0.1

x = np.arange(X_MIN, X_MAX, res_c)
y = np.arange(Y_MIN, Y_MAX, res_c)
X, Y = np.meshgrid(x, y)

cp_x = []
cp_y = []
for x_0, y_0 in product(x, y):
    x_c, y_c =  fsolve(DDT, (x_0, y_0))
    if len(cp_x) == 0:
        cp_x.append(x_c)
        cp_y.append(y_c)
    else:
        d = np.sqrt((np.array(cp_x) - x_c)**2 + (np.array(cp_x) - y_c)**2)
        if (d > epsilon).all():
            cp_x.append(x_c)
            cp_y.append(y_c)

print("Critical points found at:")            
print([i for i in zip(cp_x,cp_y)])

In [ ]:
# Creating the Grid for plotting
x = np.arange(X_MIN, X_MAX, RES)
y = np.arange(Y_MIN, Y_MAX, RES)
X, Y = np.meshgrid(x, y)

# Caculating the change vectors
dxdt = Q_SCALE*DXDT(X, Y)
dydt = Q_SCALE*DYDT(X, Y)

In [ ]:
# Plotting the phase diagram

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1,aspect='equal')

ax.axhline(0, color='.8')
ax.axvline(0, color='.8')
ax.scatter(cp_x, cp_y, s=100, color='blue', marker='o', facecolor='none')
ax.quiver(x, y, dxdt, dydt, color='blue')

ax.set_title("2D phase portrait")
ax.set_xlabel(X_LABEL)
ax.set_ylabel(Y_LABEL)

# Approximating dynamics near the critical points

In [ ]:
def linearize(x, y, dx_func, dy_func, D=0.000001):
    L = np.array(
        [[(dx_func(x+D,y) - dx_func(x-D,y)) / (2*D), (dx_func(x,y+D) - dx_func(x,y-D)) / (2*D)], 
        [(dy_func(x+D,y) - dy_func(x-D,y)) / (2*D), (dy_func(x,y+D) - dy_func(x,y-D)) / (2*D)]]
    )
    return(L)

def prettyprint_eig(M):
    vals, mat = eig(M)
    for i in range(len(vals)):
        if np.imag(vals[i]) == 0.0:
            vals = np.array(vals, dtype='float')
        print(f"\tEigenvalue {i+1} is {vals[i]:.3}, with vector ({mat[0,i]:.3}, {mat[1,i]:.3})")

In [ ]:
Ls = []
for i, (x, y) in enumerate(zip(cp_x, cp_y)):
    L = linearize(x, y, DXDT, DYDT)
    Ls.append(L)
    print(f" At critical point #{i}, ({x}, {y}), the functions are approximated by \n {L}\n")
    

In [ ]:
for i, (L, x, y) in enumerate(zip(Ls, cp_x, cp_y)):
    print(f"At critical point #{i}, ({x}, {y}),")
    prettyprint_eig(Ls[i])
    print("")

# Simulating solutions to the system 

In [ ]:
X_0 = 4 # starting x
Y_0 = 1 # starting y
T_f = 10 # final time

DT = 0.1

## Euler's method

In [ ]:
def Euler_next(x, y, dxdt, dydt, dt=DT):
    x_next = x + dxdt(x,y)*dt
    y_next = y + dydt(x,y)*dt
    return(x_next, y_next)

def simulate(x_0, y_0, t_f, simulation_function, dt=DT):
    X_solution = [x_0]
    Y_solution = [y_0]
    T = np.arange(dt, t_f, dt)
    for t in T[1:]:
        x_previous, y_previous = X_solution[-1], Y_solution[-1]
        x_next, y_next = simulation_function(x_previous, y_previous, DXDT, DYDT, dt=dt)
        X_solution.append(x_next)
        Y_solution.append(y_next)
    return (T, X_solution, Y_solution)

def simulate_collection(X_0s, Y_0s, t_f, simulation_function, dt=DT):
    X = []
    Y = []
    for X_0, Y_0 in zip(X_0s, Y_0s):
        T, X_sol, Y_sol = simulate(X_0, Y_0, t_f, simulation_function, dt=DT)
        X.append(X_sol)
        Y.append(Y_sol)
    return T, X, Y

### Simulating Euler's method

In [ ]:
T, X_euler, Y_euler = simulate(X_0, Y_0, T_f, Euler_next)

In [ ]:
X_euler[:10]

In [ ]:
Y_euler[:10]

### Graphing the simulation

In [ ]:
def plot_solution(T, X_solutions, Y_solutions, colors=['.k']):
    fig, (ax, ax_x, ax_y) = plt.subplots(figsize=(12,4), ncols=3)

    # Creating the Grid for plotting
    x = np.arange(X_MIN, X_MAX, RES)
    y = np.arange(Y_MIN, Y_MAX, RES)
    X, Y = np.meshgrid(x, y)

    # Caculating the change vectors
    dxdt = Q_SCALE*DXDT(X, Y)
    dydt = Q_SCALE*DYDT(X, Y)
    
    ax.set_aspect('equal')
    ax.axhline(0, color='.8')
    ax.axvline(0, color='.8')
    ax.scatter(cp_x, cp_y, s=100, color='blue', marker='o', facecolor='none')
    ax.quiver(x, y, dxdt, dydt, color='blue')

    ax.set_title("2D phase portrait")
    ax.set_xlabel(X_LABEL)
    ax.set_ylabel(Y_LABEL)
    ax.set_xlim([X_MIN, X_MAX])
    ax.set_ylim([Y_MIN, Y_MAX])
    
    assert len(X_solutions) == len(Y_solutions)
    if len(colors) != len(X_solutions):
        colors = len(X_solutions) * colors
    for X_solution, Y_solution, color in zip(X_solutions, Y_solutions, colors):
        ax.plot(X_solution, Y_solution, color=color)
        ax.scatter(X_solution[0], Y_solution[0], color=color, zorder=10, s=10.0)

        ax_x.set_title("{0} vs. time".format(X_LABEL))
        ax_x.plot(T, X_solution, color=color)
        ax_x.axhline(0, color='.8', zorder=-1)
        ax_x.scatter(T[0], X_solution[0], color=color, zorder=10, s=10.0)
        ax_x.set_ylabel(X_LABEL)
        ax_x.set_xlabel("Time")
        ax_x.set_ylim([X_MIN, X_MAX])

        ax_y.set_title("{0} vs. time".format(Y_LABEL))
        ax_y.plot(T, Y_solution, color=color)
        ax_y.axhline(0, color='.8', zorder=-1)
        ax_y.scatter(T[0], Y_solution[0], color=color, zorder=10, s=10.0)
        ax_y.set_ylabel(Y_LABEL)
        ax_y.set_xlabel("Time")
        ax_y.set_ylim([Y_MIN, Y_MAX])

In [ ]:
plot_solution(T, [X_euler], [Y_euler], colors=['k'])

## The midpoint method

In [ ]:
X_0 = 0 # starting x
Y_0 = 1 # starting y
T_f = 4*pi # final time

DT = 0.01

In [ ]:
def midpoint_next(x, y, dxdt, dydt, dt=DT):
    x_test = x + dxdt(x,y)*dt
    y_test = y + dydt(x,y)*dt
    
    x_next = x + dxdt((x + x_test)/2., (y + y_test)/2)*dt
    y_next = y + dydt((x + x_test)/2., (y + y_test)/2)*dt
    
    return(x_next, y_next)

### Simulating 

In [ ]:
T, X_euler, Y_euler = simulate(X_0, Y_0, T_f, Euler_next)
T, X_mid, Y_mid = simulate(X_0, Y_0, T_f, midpoint_next)

### Graphing the simulation

In [ ]:
plot_solution(T, [X_mid], [Y_mid], colors=['b'])

## Comparing Midpoint method with Euler method

In [ ]:
plot_solution(T, [X_euler, X_mid], [Y_euler, Y_mid], colors=['k', 'b'])

## Plotting a series of trajectories

In [ ]:
X_MIN = -6
X_MAX = 6
Y_MIN = -6
Y_MAX = 6

X_0s = np.arange(-6, 8, 1)
Y_0s = len(X_0s)*[-1]
T_f = 2

In [ ]:
T, X, Y = simulate_collection(X_0s, Y_0s, T_f, midpoint_next)
colors = [cm.cool(i / float(len(X))) for i in range(len(X))]
plot_solution(T, X, Y, colors=colors)